In [1]:
import torch
import helper_funcs
from sentence_transformers import SentenceTransformer
from sentence_transformers.cross_encoder import CrossEncoder

encoder = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

/home/mputo/.local/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
collection = helper_funcs.tsv_to_df("../collectionandqueries/collection", col_name=["id", "paragraph"], index_col="id")

In [3]:
queries = helper_funcs.tsv_to_df("../collectionandqueries/queries.train", col_name=["query number", "query text"], index_col="query number")

In [4]:
r = helper_funcs.create_redis()

helper_funcs.save_collection(r, collection, 1000)

In [5]:
d = 384
description = "Flat"

index = helper_funcs.create_index(d, description, gpu=False)

In [6]:
xb = torch.load('../msmarco-vec/msmarco-vectors-1.pt')

In [7]:
helper_funcs.populate_index(xb, index, to_cpu=True)

print(index.ntotal)

1000000


In [11]:
import time

In [12]:
for i in range(0, 5):
    start = time.time()
    for query_number, query_text in queries.head(10).iterrows():
        x = helper_funcs.search_and_rerank_redis(query_text[0], 10, encoder=encoder, cross_encoder=cross_encoder, index=index, rds=r)
    end = time.time()

    print(f"{i}. Time elapsed: ", end - start)

0. Time elapsed:  3.4585072994232178
1. Time elapsed:  1.8585867881774902
2. Time elapsed:  2.1940665245056152
3. Time elapsed:  1.8644893169403076
4. Time elapsed:  1.8489837646484375


In [13]:
for i in range(0, 5):
    start = time.time()
    for query_number, query_text in queries.head(10).iterrows():
        x = helper_funcs.search_and_rerank(query_text[0], 10, encoder=encoder, cross_encoder=cross_encoder, index=index, collection=collection)
    end = time.time()

    print(f"{i}. Time elapsed: ", end - start)

0. Time elapsed:  110.79460740089417
1. Time elapsed:  96.4015302658081
2. Time elapsed:  96.12292861938477
3. Time elapsed:  94.7905855178833
4. Time elapsed:  94.7287974357605
